# Gruppe 6 - Abgabe 1 & 2

In [ ]:
import pandas as pd
import numpy as np
import pickle as pickle
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn import utils
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from skimage.filters import threshold_mean
from skimage.feature import hog
from sklearn.decomposition import PCA
from skimage.color import rgb2gray
from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage import img_as_ubyte


: 

In [ ]:
calcSVM = True;
calcNN = True;

: 

## Import Data

In [ ]:
# Load Data
db1 = pd.read_pickle(r'.\cifar-10-batches-py\data_batch_1')
db2 = pd.read_pickle(r'.\cifar-10-batches-py\data_batch_2')
db3 = pd.read_pickle(r'.\cifar-10-batches-py\data_batch_3')
db4 = pd.read_pickle(r'.\cifar-10-batches-py\data_batch_4')
db5 = pd.read_pickle(r'.\cifar-10-batches-py\data_batch_5')
dbt = pd.read_pickle(r'.\cifar-10-batches-py\test_batch')
meta = pd.read_pickle(r'.\cifar-10-batches-py\batches.meta')

labels = db1['labels'] + db2['labels'] + db3['labels'] + db4['labels'] + db5['labels'] + dbt['labels']

data = np.vstack((db1['data'], db2['data'], db3['data'], db4['data'], db5['data'], dbt['data']))

len(data)

: 

In [ ]:
# label names
labelNames = meta['label_names']

#reshape
data = np.array(data)
data = data.reshape(len(data), 3, 32, 32).transpose(0,2,3,1)

: 

## Feature Engeneering

In [ ]:
data_features = []
pca = PCA(n_components=32) # 32 x components

for i in range(0, 60000):
    img = data[i]
    arr = np.array([])

    gray_img = rgb2gray(img) #remove color --> to gray
    x = threshold_mean(gray_img) # calculate threshold
    binary = gray_img > x #check if values above or below thershold
    binary_fit = pca.fit(binary) # reduce dimension

    data_feature = (np.array(binary_fit.components_).ravel())

    #gray_ubytes = img_as_ubyte(gray_img)
    #data_feature = np.append(data_feature, entropy(gray_ubytes, disk(5))) #calculate histogram

    fd, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, channel_axis=-1)

    data_feature = np.append(data_feature, fd)

    data_features.append(data_feature)

: 

In [ ]:
#components
len(data_features[0])

: 

In [ ]:
#show for test
index = 0
plt.imshow(data[index], cmap='gray')
print(labelNames[labels[index]])

: 

## Prepare Data

In [ ]:
#shuffle
labels, data_features, data= utils.shuffle(labels, data_features, data)

: 

In [ ]:
cntTrain = 45000
cntTest = 15000

# splitt
X_train = np.array(data[:cntTrain])
Y_train = np.array(labels[:cntTrain])

X_test = np.array(data[-cntTest:])
Y_test = np.array(labels[-cntTest:])

Y_train = Y_train.flatten()
Y_test = Y_test.flatten()

X_train_features = data_features[:cntTrain]
Y_train_features = labels[:cntTrain]

X_test_features = data_features[-cntTest:]
Y_test_features = labels[-cntTest:]

: 

## Vizualize Data

In [ ]:
plt.figure(figsize=(10,7))
p = sns.countplot(Y_train.flatten())
p.set(xticklabels=labelNames)

: 

## SVM

In [ ]:
if(calcSVM):
    
    Y_test_features = np.array(Y_test_features)

    # Create the SVM
    svm = SVC(random_state=42, kernel='rbf') # rbf or poly

    # Fit the data to the SVM classifier
    svm = svm.fit(X_train_features, Y_train_features)

    # Generate predictions
    y_pred_features = svm.predict(X_test_features)

    # Evaluate by means of accuracy
    accuracy = accuracy_score(Y_test, y_pred_features)
    print(f'Model accuracy: {accuracy}')

    vscores = cross_val_score(svm, X_test_features, Y_test_features, cv=5)
    print(vscores)


: 

In [ ]:
if(calcSVM):
    #evaluate with confusion matrix
    cf_matrix = confusion_matrix(Y_test, y_pred_features)

    plt.figure(figsize=(12, 9))
    c = sns.heatmap(cf_matrix, annot=True, fmt='g', cmap="crest")
    c.set(xticklabels=labelNames, yticklabels=labelNames)

: 

## Neuronale Network

In [ ]:
if(calcNN):
input_shape = (32, 32, 3)

X_train=X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 3)
X_train=X_train / 255.0
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 3)
X_test=X_test / 255.0

Y_train = tf.one_hot(Y_train.astype(np.int32), depth=10)
Y_test = tf.one_hot(Y_test.astype(np.int32), depth=10)

: 

In [ ]:
if(calcNN):
batch_size = 32
num_classes = 10
epochs = 100

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', input_shape=X_train.shape[1:], activation='relu'),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax'),
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00001),
            loss='categorical_crossentropy', metrics=['acc'])

: 

In [ ]:
if(calcNN):
history = model.fit(X_train, Y_train, batch_size=batch_size,
                epochs=epochs)

: 

In [ ]:
if(calcNN):
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training Loss")
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['acc'], color='b', label="Training Accuracy")
legend = ax[1].legend(loc='best', shadow=True)

: 

In [ ]:
if(calcNN):
    test_loss, test_acc = model.evaluate(X_test, Y_test)

: 

In [ ]:
if(calcNN):
    # Predict the values from the validation dataset
    y_pred = model.predict(X_test)
    # Convert predictions classes to one hot vectors 
    y_pred_classes = np.argmax(y_pred,axis = 1) 
    # Convert validation observations to one hot vectors
    y_true = np.argmax(Y_test,axis = 1)
    # compute the confusion matrix
    confusion_mtx = tf.math.confusion_matrix(y_true, y_pred_classes)

    plt.figure(figsize=(12, 9))
    c = sns.heatmap(confusion_mtx, annot=True, fmt='g', cmap="crest")
    c.set(xticklabels=labelNames, yticklabels=labelNames)

: 